In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import time

# 1. Загрузка и подготовка данных
df = pd.read_csv('day.csv', parse_dates=['dteday'], index_col='dteday')

# Исключаем нерелевантные признаки
excluded_features = ['instant', 'casual', 'registered', 'cnt']
features = [col for col in df.columns if col not in excluded_features]
target = 'cnt'

# 2. Временное разделение данных (без перемешивания)
train_size = int(0.8 * len(df))
train_df, test_df = df.iloc[:train_size], df.iloc[train_size:]

# 3. Нормализация с сохранением временного порядка
scaler = StandardScaler()
X_train = scaler.fit_transform(train_df[features])
X_test = scaler.transform(test_df[features])

y_train = train_df[target].values.reshape(-1, 1)
y_test = test_df[target].values.reshape(-1, 1)

# 4. Создание временных последовательностей с недельным окном
def create_sequences(data, targets, window_size=7):
    X_seq, y_seq = [], []
    for i in range(len(data)-window_size):
        X_seq.append(data[i:i+window_size])
        y_seq.append(targets[i+window_size])
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequences(X_train, y_train)
X_test_seq, y_test_seq = create_sequences(X_test, y_test)

# 5. Архитектура модели с LSTM
model = Sequential([
    LSTM(128, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2]), 
        return_sequences=True, dropout=0.2),
    LSTM(64, dropout=0.2, recurrent_dropout=0.1),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse'])

# 6. Обучение модели
start_time = time.time()
history = model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    shuffle=False,
    verbose=1
)
training_time = time.time() - start_time

# 7. Оценка производительности
test_results = model.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f'''
Результаты тестирования:
Время обучения: {training_time:.2f} сек
Средняя абсолютная ошибка (MAE): {test_results[1]:.4f}
Среднеквадратичная ошибка (MSE): {test_results[2]:.4f}
''')


Epoch 1/100
17/17 [==============================] - 76s 534ms/step - loss: 18003802.0000 - mae: 3915.5432 - mse: 18003802.0000 - val_loss: 44539252.0000 - val_mae: 6621.8345 - val_mse: 44539252.0000
Epoch 2/100
17/17 [==============================] - 1s 44ms/step - loss: 17996730.0000 - mae: 3914.6248 - mse: 17996730.0000 - val_loss: 44523768.0000 - val_mae: 6620.6714 - val_mse: 44523768.0000
Epoch 3/100
17/17 [==============================] - 1s 43ms/step - loss: 17976154.0000 - mae: 3912.0388 - mse: 17976154.0000 - val_loss: 44483488.0000 - val_mae: 6617.6333 - val_mse: 44483488.0000
Epoch 4/100
17/17 [==============================] - 1s 43ms/step - loss: 17945130.0000 - mae: 3908.1763 - mse: 17945130.0000 - val_loss: 44428960.0000 - val_mae: 6613.5044 - val_mse: 44428960.0000
Epoch 5/100
17/17 [==============================] - 1s 44ms/step - loss: 17913528.0000 - mae: 3904.1360 - mse: 17913528.0000 - val_loss: 44349388.0000 - val_mae: 6607.5054 - val_mse: 44349388.0000
Epoch 6/